# Introduction
This notebook demonstrates a bidirectional integration between the LAMMPS molecular dynamics engine and a real-time visualization pipeline based on ANARI, using Ascent for in situ data streaming. The goal is to enable interactive exploration and control of scientific simulations by coupling numerical state with a rendering backend. The pipeline supports real-time feedback, allowing users to inspect and manipulate simulation parameters during runtime.

## Set Environment

In [1]:
import os
import copy

# Store a deep copy of the original environment
original_env = copy.deepcopy(os.environ)


In [19]:
# Reset os.environ to its original state
os.environ.clear()
os.environ.update(original_env)

In [7]:
from dotenv import dotenv_values
import os

def append_env_from_files(*env_files):
    """
    Load and append environment variables from one or more .env files.
    If a variable already exists, prepend the new value unless it's already included.
    """
    for file in env_files:
        env_vars = dotenv_values(file)
        for key, new_val in env_vars.items():
            current_val = os.environ.get(key, "")
            if current_val:
                if new_val not in current_val.split(":"):
                    os.environ[key] = f"{new_val}:{current_val}"
            else:
                os.environ[key] = new_val

# LAMMPS
LAMMPS (Large-scale Atomic/Molecular Massively Parallel Simulator) is a classical molecular dynamics code developed for parallel simulations of particles interacting under various force fields. It supports a wide range of models for solid-state materials, soft matter, and coarse-grained systems, and can be extended via user-defined fixes and external control interfaces.

## Load Variables

In [21]:
append_env_from_files("python_vars/mpich_vars")
append_env_from_files("python_vars/zlib_vars")
append_env_from_files("python_vars/hdf5_vars")
append_env_from_files("python_vars/lammps_vars")

## Compile and Run

In [43]:
!examples/lammps/simple/build_lammps.sh

In [53]:
!examples/lammps/simple/run.sh &> /dev/null

## Visualize LAMMPS Output

In [51]:
!ffmpeg -y -i lammps_box.mpg -c:v libx264 lammps_box.mp4 &> /dev/null


In [ ]:
from IPython.display import Video
Video("lammps_box.mp4")

# Ascent
Ascent is an in situ visualization and analysis library designed for high-performance computing workloads. It uses the Conduit data model to receive simulation state and supports multiple rendering backends through a high-level action interface. 

Ascent is used in this workflow to extract, convert, and transmit LAMMPS simulation data to downstream visualization components without disrupting the simulation loop.

## Load Variables

In [61]:
append_env_from_files("python_vars/ascent_anari_vars")
append_env_from_files("python_vars/conduit_vars")

In [62]:
!echo $LIBRARY_PATH

/home/droaperdomo/ascent_anari/obj_bin/install/conduit-v0.9.4/lib:/home/droaperdomo/ascent_anari/obj_bin/install/conduit-v0.9.4/lib64:/home/droaperdomo/ascent_anari/obj_bin/install/ascent-checkout/lib:/home/droaperdomo/ascent_anari/obj_bin/install/ascent-checkout/lib64:/home/droaperdomo/ascent_anari/obj_bin/install/conduit-v0.9.4/lib:/home/droaperdomo/ascent_anari/obj_bin/install/conduit-v0.9.4/lib64:/home/droaperdomo/ascent_anari/obj_bin/install/ascent-checkout/lib:/home/droaperdomo/ascent_anari/obj_bin/install/ascent-checkout/lib64:/home/droaperdomo/lammps/obj_bin/lib:/home/droaperdomo/lammps/obj_bin/lib64:/home/droaperdomo/tools/hdf5_bin/lib:/home/droaperdomo/tools/hdf5_bin/lib64:/home/droaperdomo/tools/zlib_bin/lib:/home/droaperdomo/tools/zlib_bin/lib64:/home/droaperdomo/tools/mpich/lib:/home/droaperdomo/tools/mpich/lib64


In [64]:
!examples/lammps/simple/build_lammps_ascent.sh

/usr/bin/ld: /tmp/ccHCSAwS.o: in function `lammps_atoms_to_conduit_mesh(LAMMPS_NS::LAMMPS*, conduit::Node&)':
lammps_mpi_ascent.cpp:(.text+0x55c): undefined reference to `conduit::Node::operator[](std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > const&)'
/usr/bin/ld: lammps_mpi_ascent.cpp:(.text+0x571): undefined reference to `conduit::Node::operator=(char const*)'
/usr/bin/ld: lammps_mpi_ascent.cpp:(.text+0x5d0): undefined reference to `conduit::Node::operator[](std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > const&)'
/usr/bin/ld: lammps_mpi_ascent.cpp:(.text+0x602): undefined reference to `conduit::Node::set_external(float*, long, long, long, long, long)'
/usr/bin/ld: lammps_mpi_ascent.cpp:(.text+0x662): undefined reference to `conduit::Node::operator[](std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > const&)'
/usr/bin/ld: lammps_mpi_ascent.cpp:(.text+0x694): undefined reference to `conduit:

# ANARI
ANARI (Analytic Rendering Interface) is a rendering API that provide type-safe wrappers that abstracts scene description and rendering tasks across heterogeneous devices. It enables portable rendering by decoupling application logic from graphics backends and supports in-memory hierarchical object trees to describe full frame content, including surface geometry and volumetric data.

ANARI provides semantics for rendering engines to implement extensions, support asynchronous scene updates, and access zero-copy memory, facilitating interactive and low-latency visualization. In this integration, ANARI is used to render particle and field data exported from LAMMPS via Ascent.

# ANARI-USD
ANARI-USD is an implementation of an ANARI rendering device that outputs data in the USD (Universal Scene Description) format. It allows ANARI applications to render USD scene graphs directly or engage with rendering pipelines to visualize ANARI content using USD-compatible tools and frameworks, including NVIDIA’s Omniverse.

This backend is used to stream LAMMPS state into a USD scene for rendering and visualization.


In [ ]:
source ~/tools/mpich_vars.sh
source ~/tools/onetbb_vars.sh
source ~/tools/opensubdiv_vars.sh 
source ~/tools/glfw_vars.sh
source ~/tools/openusd_vars.sh
source ~/tools/anarisdk_vars.sh
source ~/tools/openusd_vars.sh
source ~/tools/connectsamples_vars.sh

# Omniverse
Omniverse is a multi-GPU, collaborative simulation and visualization platform built around USD. It provides infrastructure for real-time rendering, physics simulation, and interaction. In this workflow, Omniverse acts as the visualization environment that consumes the USD scene generated by ANARI-USD, allowing immersive inspection and interaction with the evolving simulation state.